In [1]:
from __future__ import print_function, division

In [265]:
import json
import requests
from lxml import html
from bs4 import BeautifulSoup
from collections import OrderedDict
import argparse
import re
import dateutil.parser
import pandas as pd
import pymysql


In [219]:
# if needed: pip install requests
url = 'https://seattle.craigslist.org/search/bia'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page,"lxml")

In [226]:
total = soup.find('span', {'class': 'totalcount'})
max = int(total.text)

In [220]:
# titles = soup.findAll('a', {'class': 'result-title'})
# for title in titles:
#     print(title.text)

In [244]:
dicts = {}
i = 0
s = 0
while s < max:
    url = 'https://seattle.craigslist.org/search/bia?s='+str(s)
    print(url)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    for ultag in soup.find_all('ul', {'class': 'rows'}):
        for litag in ultag.find_all('li'):
            dicts[i] = {}
            title = litag.find('a', {'class': 'result-title'})
            price = litag.find('span', {'class': 'result-price'})
            date = litag.find('time',{'class': 'result-date'} )
            dicts[i]['title'] = title.text
            dicts[i]['date'] = date['datetime']
            if price:
                dicts[i]['price'] = price.text
            i+=1
    s+=120
        
        

https://seattle.craigslist.org/search/bia?s=0
https://seattle.craigslist.org/search/bia?s=120
https://seattle.craigslist.org/search/bia?s=240
https://seattle.craigslist.org/search/bia?s=360
https://seattle.craigslist.org/search/bia?s=480
https://seattle.craigslist.org/search/bia?s=600
https://seattle.craigslist.org/search/bia?s=720
https://seattle.craigslist.org/search/bia?s=840
https://seattle.craigslist.org/search/bia?s=960
https://seattle.craigslist.org/search/bia?s=1080
https://seattle.craigslist.org/search/bia?s=1200
https://seattle.craigslist.org/search/bia?s=1320
https://seattle.craigslist.org/search/bia?s=1440
https://seattle.craigslist.org/search/bia?s=1560
https://seattle.craigslist.org/search/bia?s=1680
https://seattle.craigslist.org/search/bia?s=1800
https://seattle.craigslist.org/search/bia?s=1920
https://seattle.craigslist.org/search/bia?s=2040
https://seattle.craigslist.org/search/bia?s=2160
https://seattle.craigslist.org/search/bia?s=2280
https://seattle.craigslist.org/

In [248]:
df = pd.DataFrame(dicts).T

In [264]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2811 entries, 0 to 2810
Data columns (total 3 columns):
date     2811 non-null datetime64[ns]
price    2753 non-null float64
title    2811 non-null object
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 87.8+ KB


In [258]:
df.head()

,date,price,title
0,2018-04-16 14:50:00,200,Hard shell bike travel case with rollers
1,2018-04-16 14:50:00,1,Adolescent Group Home is desperately seeking 4...
2,2018-04-16 14:50:00,250,Vintage Schwinn World Sport Road Bike
3,2018-04-16 14:50:00,550,Schwinn Cruiser Deluxe 26 inch Bicycle
4,2018-04-16 14:49:00,150,Hercules Magnum Bike made by Sachs


In [253]:
df['date'] = pd.to_datetime(df['date'])

In [257]:
df['price'] = df['price'].str.replace('$', '')

In [263]:
df['price'] = pd.to_numeric(df['price'])

In [266]:
#insert into RDS
host="metis.cmwyee1fz5ce.us-east-2.rds.amazonaws.com"
port=3306
dbname="metis"
user="mmidzik"
password="contigo434"

conn = pymysql.connect(host, user=user,port=port,
                           passwd=password, db=dbname)

In [273]:
df.to_sql('cl', con=conn, schema = 'metis', flavor='mysql',if_exists='append')

ValueError: database flavor mysql is not supported